# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
#pip install chart-studio

In [2]:
#pip install cufflinks

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [49]:
import pandas as pd
import chart_studio.plotly as py
import plotly.express as px
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [59]:
df = pd.read_excel('../data/Online Retail.xlsx')

df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [6]:
#Filtro con las condiciones
df_wo_uk = df[(df['Country'] != 'United Kingdom') & (df['InvoiceDate'] >= '2011-04-01') &  (df['InvoiceDate'] < '2011-05-01')]
df_wo_uk.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
724,549667,2011-04-11 12:20:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,14911,EIRE
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France
3320,550899,2011-04-21 12:07:00,22752,SET 7 BABUSHKA NESTING BOXES,2,8.50,17.00,13505,Switzerland
4880,550527,2011-04-19 10:48:00,84879,ASSORTED COLOUR BIRD ORNAMENT,40,1.69,67.60,12476,Germany
4882,550620,2011-04-19 13:39:00,84879,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,54.08,12585,Germany


In [7]:
df_wo_uk = df_wo_uk.groupby('Country')[['Quantity', 'Revenue']].sum().head()

In [8]:
# otra manera
# df_wo_uk = df_wo_uk.groupby('Country').agg({'Quantity': 'sum', 'Revenue': 'sum'}).head() 

In [9]:
df_wo_uk.iplot(kind='bar', xTitle='Country', title='Total quantities and revenues by country, except UK')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [10]:
df_france_2011 = df[(df['Country'] == 'France') & (df['InvoiceDate'] >= '2011-01-01') &  (df['InvoiceDate'] < '2011-06-01')]
df_france_2011.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France
2645,540976,2011-01-12 15:00:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3.75,15.00,12652,France
3290,545181,2011-02-28 14:45:00,22752,SET 7 BABUSHKA NESTING BOXES,4,8.50,34.00,12509,France
4614,542629,2011-01-31 09:57:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France
4795,548409,2011-03-31 10:27:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France


In [11]:
dfrance = df_france_2011.groupby('InvoiceDate')[['Quantity','Revenue']].sum()
dfrance

,Quantity,Revenue
InvoiceDate,,
2011-01-05 12:42:00,359,502.07
2011-01-05 14:48:00,369,763.11
2011-01-06 14:26:00,261,283.77
2011-01-06 16:12:00,177,425.25
2011-01-07 12:07:00,488,676.16
...,...,...
2011-05-25 12:54:00,374,514.74
2011-05-26 14:53:00,86,196.50
2011-05-26 17:16:00,424,362.00


In [12]:
dfrance.iplot(kind='line', xTitle='France', title='Total quantities & revenues in France')

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [13]:
df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [14]:
pb = df.query('Description == "PARTY BUNTING"')
pb.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
252864,536956,2010-12-03 12:43:00,47566,PARTY BUNTING,5,4.65,23.25,14210,United Kingdom
252865,537065,2010-12-05 11:57:00,47566,PARTY BUNTING,5,4.65,23.25,12567,France
252866,537128,2010-12-05 12:15:00,47566,PARTY BUNTING,2,4.65,9.30,12841,United Kingdom
252867,537142,2010-12-05 12:57:00,47566,PARTY BUNTING,1,4.65,4.65,12748,United Kingdom
252868,537420,2010-12-06 15:18:00,47566,PARTY BUNTING,5,4.65,23.25,17519,United Kingdom


In [15]:
df[df["Description"] == "PARTY BUNTING"] #otra manera

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
252864,536956,2010-12-03 12:43:00,47566,PARTY BUNTING,5,4.65,23.25,14210,United Kingdom
252865,537065,2010-12-05 11:57:00,47566,PARTY BUNTING,5,4.65,23.25,12567,France
252866,537128,2010-12-05 12:15:00,47566,PARTY BUNTING,2,4.65,9.30,12841,United Kingdom
252867,537142,2010-12-05 12:57:00,47566,PARTY BUNTING,1,4.65,4.65,12748,United Kingdom
252868,537420,2010-12-06 15:18:00,47566,PARTY BUNTING,5,4.65,23.25,17519,United Kingdom
...,...,...,...,...,...,...,...,...,...
254255,580937,2011-12-06 14:13:00,47566,PARTY BUNTING,4,4.95,19.80,16143,United Kingdom
254256,581142,2011-12-07 13:38:00,47566,PARTY BUNTING,3,4.95,14.85,14204,United Kingdom
254257,581345,2011-12-08 12:15:00,47566,PARTY BUNTING,5,4.95,24.75,17315,United Kingdom
254258,581417,2011-12-08 15:01:00,47566,PARTY BUNTING,8,4.95,39.60,13013,United Kingdom


In [30]:
pb = pb.groupby('Country').mean(numeric_only=True)
pb.head()

,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
Country,,,,,
Australia,554329.625000,33.125000,4.7125,143.78125,12399.250000
Austria,552202.000000,8.000000,4.9500,39.60000,12414.000000
Belgium,557600.000000,4.000000,4.9500,19.80000,12363.000000
Channel Islands,561821.666667,13.333333,4.9500,66.00000,14934.000000
Cyprus,553141.333333,2.333333,4.7500,10.95000,12373.333333


In [31]:
pb2 = pb[['Quantity','UnitPrice']]
pb2.reset_index(inplace=True)
pb2.head()

,Country,Quantity,UnitPrice
0,Australia,33.125000,4.7125
1,Austria,8.000000,4.9500
2,Belgium,4.000000,4.9500
3,Channel Islands,13.333333,4.9500
4,Cyprus,2.333333,4.7500


In [32]:
pb2.iplot(x='Quantity',
            y='UnitPrice',
            categories='Country',
            xTitle='Avg quantity', 
            yTitle='Avg unitPrice', 
            title='Relationship between avg quantity and avg unit price for PARTY BUNTING',
            color='blue'
            );


## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [57]:
pb.head()

,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
Country,,,,,
Australia,554329.625000,33.125000,4.7125,143.78125,12399.250000
Austria,552202.000000,8.000000,4.9500,39.60000,12414.000000
Belgium,557600.000000,4.000000,4.9500,19.80000,12363.000000
Channel Islands,561821.666667,13.333333,4.9500,66.00000,14934.000000
Cyprus,553141.333333,2.333333,4.7500,10.95000,12373.333333


In [87]:
eire = df[df.Country == 'EIRE'] 

eire = pd.pivot_table(germany, values=['Quantity'], index='InvoiceNo', aggfunc='sum')

eire.iplot(kind='bar', xTitle='invoice', yTitle='qty', 
           title='Ireland quantity per invoice dist')

In [95]:
germany = df[df.Country == 'Germany'] 

germany = pd.pivot_table(germany, values=['Quantity'], index='InvoiceNo', aggfunc='sum')

germany.iplot(kind='bar', xTitle='invoice', yTitle='qty', 
           title='Germany quantity per invoice dist')

In [97]:
france = df[df.Country == 'France'] 

france = pd.pivot_table(germany, values=['Quantity'], index='InvoiceNo', aggfunc='sum')

france.iplot(kind='bar', xTitle='invoice', yTitle='qty', 
           title='France quantity per invoice dist')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [68]:
product_list = ['JUMBO BAG RED RETROSPOT',
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [75]:
df5 = df[(df['Description'].isin(product_list)) & (df['Country'].isin(country_list))]

df5 = df5.pivot_table(values='Revenue', 
                      columns='Description',
                      index='Country', 
                      aggfunc='sum')

df5.reset_index(inplace=True)


df5.iplot(kind='bar', 
           x='Country', 
           title='Revenue by Country',
           xTitle='Country', 
           yTitle='Revenue')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.